# Creating a Heatmap of Videos Uploaded During the Riots

## Setup

### Imports

In [1]:
from datetime import date, datetime, timedelta, tzinfo

import geopandas
import folium
import pandas as pd
from folium import plugins as folium_plugins

### Helpers

In [2]:
def load_geodata(dc_attack=False):
    """ Return DF """
    geodata_path = 'data/parler-videos-geocoded.csv'
    df_geodata = pd.read_csv(geodata_path)
    df_geodata['Timestamp'] = df_geodata['Timestamp'].map(lambda x: datetime.fromisoformat(x))
    
    if dc_attack:
        df_geodata = df_geodata[df_geodata['Timestamp'].map(lambda x: x.year) == 2021]
        df_geodata = df_geodata[df_geodata['Timestamp'].map(lambda x: x.month) == 1]
        df_geodata = df_geodata[df_geodata['Timestamp'].map(lambda x: x.day) == 6]
        df_geodata = df_geodata[
            (df_geodata['Longitude'] >= -77.6349) & \
            (df_geodata['Longitude'] <= -76.4803) & \
            (df_geodata['Latitude'] >= 38.5572) & \
            (df_geodata['Latitude'] <= 39.2300)]
    
    df_geodata = df_geodata.set_index('Timestamp')
    df_geodata = df_geodata.tz_localize('UTC').tz_convert('EST')
        
    print(f'Loaded {len(df_geodata)} data points.')
    return df_geodata

## Creating the Map

### Loading Data

In [3]:
df_dc_riots = load_geodata(dc_attack=True)

Loaded 1209 data points.


### Grouping in 20 Minute Chunks

In [4]:
time_increment = timedelta(minutes=20)
start_time = datetime(2021, 1, 6, hour=10, tzinfo=df_dc_riots.index[0].tz)
end_time = datetime(2021, 1, 6, hour=18, tzinfo=df_dc_riots.index[0].tz)

In [5]:
heat_data = []
time_index = []

current_time = start_time
while current_time <= end_time - time_increment:
    window_open = current_time
    window_close = current_time + time_increment
    
    time_window_data = [
        [row['Latitude'], row['Longitude']]
        for index, row in df_dc_riots[
            df_dc_riots.index.to_series().apply(lambda x: window_open <= x < window_close)
        ].iterrows()
    ]
    
    heat_data.append(time_window_data)
    time_index.append(current_time.strftime("%d. %B %Y %H:%M") + '-' + window_close.strftime("%H:%M"))
    
    current_time = window_close

### Creating the Map

In [6]:
heatmap_dc = folium.Map(location=[38.89, -77.03], tiles="cartodbpositron", zoom_start=14)

hm = folium_plugins.HeatMapWithTime(
    heat_data,
    index=time_index,
    radius=5,
    gradient={0.5: 'blue', 0.75: 'lime', 1.0: 'red'},
    auto_play=True,
    max_opacity=0.8
)

hm.add_to(heatmap_dc)
heatmap_dc.save("heatmap_dc_2021_01_06.html")
heatmap_dc